# Tugas 7 **Keyword Extraction**

Nama : Zainal Arifin

NIM  : 220411100103


#**Install Library**


In [1]:
!pip install selenium
!pip install networkx
!pip install matplotlib
!pip install nltk
!pip install tqdm
!pip install pandas
!pip install sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 11.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd
import re

from tqdm import tqdm

import pickle
import nltk
import networkx as nx
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from collections import Counter
from itertools import combinations
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Crawling Data Artikel Berita
melakukan crawling untuk mengambil judul, isi, tanggal, dan kategori artikel dari URL yang diberikan, dan menyajikan data tersebut dalam bentuk DataFrame untuk kemudahan analisis lebih lanjut.

In [3]:
def web_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def get_element_text(driver, xpath):
    try:
        return WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text.strip()
    except Exception as e:
        print(f"Error finding element with XPath {xpath}: {e}")
        return ""

def extract_article_content(driver, article_url):
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1'))
    )
    title = get_element_text(driver, './/h1')

    # XPath untuk tanggal
    date = get_element_text(driver, './/p[@class="pt-20 date"]')

    # XPath untuk konten
    content_elements = driver.find_elements(By.XPATH, './/div[@class="news-text"]/p')
    content = " ".join(p.text for p in content_elements)

    # XPath untuk kategori
    kategori = get_element_text(driver, './/div[@class="breadcrumb-content"]/p')

    return {
        "Title": title,
        "Date": date,
        "Content": content,
        "Category": kategori  # Menyimpan kategori dalam hasil
    }

driver = web_driver()

# Meminta input dari pengguna untuk URL artikel
url_input = input("Masukkan URL artikel: ")

# Scraping data dari URL artikel yang diberikan oleh pengguna
article_data = extract_article_content(driver, url_input)

driver.quit()

# Simpan hasil scraping ke dalam DataFrame
data = pd.DataFrame([article_data])  # Mengonversi hasil ke dalam list untuk membuat DataFrame

Masukkan URL artikel: https://www.metrotvnews.com/read/kpLCWVx8-menpora-buka-kasad-6th-asian-taekwondo-open-championship-2024


PREPROCESSING

In [5]:
# Fungsi untuk mengubah teks menjadi huruf kecil
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

data['lower case'] = data['Content'].apply(clean_lower)

# Fungsi untuk membersihkan tanda baca dan angka (menghapus koma)
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z .]+')  # Koma dihilangkan dari pengecualian
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

data['tanda baca'] = data['lower case'].apply(clean_punct)

# Fungsi untuk normalisasi spasi
def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

data['spasi'] = data['tanda baca'].apply(_normalize_whitespace)

# Tokenisasi
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return text

data['token'] = data['spasi'].apply(tokenize_text)

# Pembersihan stopwords setelah tokenisasi
def clean_stopwords(tokens):
    if isinstance(tokens, list):
        stopword = set(stopwords.words('indonesian'))
        filtered_tokens = [word for word in tokens if word not in stopword]
        return filtered_tokens
    return tokens

data['stopwords'] = data['token'].apply(clean_stopwords)

# Gabungkan kembali token menjadi string
data['processed_text'] = data['stopwords'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else '')

Menyimpan Kalimat dan Kata

In [6]:
# Membuat kumpulan kalimat
sentences = []
for p in data['processed_text']:
  sentences.extend(sent_tokenize(p))

# Membuat kumpulan kata unik
vocabulary = set()
for sentence in sentences:
  for word in sentence.split():
    vocabulary.add(word)

print(sentences)
print(vocabulary)

['tangerang menteri pemuda olahraga republik indonesia menpora ri dito ariotedjo resmi membuka kejuaraan taekwondo tingkat asia bertajuk kasad th asian taekwondo open championship sport center indoor tangerang banten selasa siang .', 'tama kemenpora mengapresiasi ketua pb ti keras menggelar kejuaraan tingkat asia diadakan .', 'negara berpartisipasi menpora dito membuka kejuaraan .', 'menpora dito menilai cabor taekwondo memiliki peran membentuk karakter pemuda bangsa .', 'indonesia menpora dito taekwondo berkembang pesat ribu pegiat berprestasi tingkat internasional .', 'taekwondo memiliki peran membentuk generasi muda tangguh disiplin berjiwa advertisement cabor taekwondo asian games cina menyumbangkan medali emas perunggu .', 'tingkat negeri taekwondo berkembang pesat .', 'ribu praktisi taekwondo dilatih ribu pelatih berlisensi mengantarkan generasi muda prestasi tingkat regional global urainya .', 'selamat bertanding atlet tunjukkan semangat dedikasi sportivitas semoga kejuaraan ins

Buat Dataframe

In [8]:
pd.set_option('future.no_silent_downcasting', True)
df = pd.DataFrame(columns=list(vocabulary), index=sentences)

# Mengisi nilai kosong dengan 0
df = df.fillna(0)  # Fill with 0s

# Menghitung nilai setiap kata pada kalimat
for i, sentence in enumerate(sentences):
  for word in sentence.split():
    df.loc[sentence, word] += 1

# Melihat isi Dataframe
df

,diadakan,timor,brunei,berpartisipasi,republik,center,advertisement,noc,hongkong,internasional,...,selasa,dedikasi,bertanding,ribu,sapta,global,nepal,korea,netral,medali
tangerang menteri pemuda olahraga republik indonesia menpora ri dito ariotedjo resmi membuka kejuaraan taekwondo tingkat asia bertajuk kasad th asian taekwondo open championship sport center indoor tangerang banten selasa siang .,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
tama kemenpora mengapresiasi ketua pb ti keras menggelar kejuaraan tingkat asia diadakan .,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
negara berpartisipasi menpora dito membuka kejuaraan .,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
menpora dito menilai cabor taekwondo memiliki peran membentuk karakter pemuda bangsa .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
indonesia menpora dito taekwondo berkembang pesat ribu pegiat berprestasi tingkat internasional .,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
taekwondo memiliki peran membentuk generasi muda tangguh disiplin berjiwa advertisement cabor taekwondo asian games cina menyumbangkan medali emas perunggu .,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
tingkat negeri taekwondo berkembang pesat .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ribu praktisi taekwondo dilatih ribu pelatih berlisensi mengantarkan generasi muda prestasi tingkat regional global urainya .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
selamat bertanding atlet tunjukkan semangat dedikasi sportivitas semoga kejuaraan inspirasi prestasi pungkas menpora dito didampingi deputi peningkatan prestasi olahraga surono .,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
kejuaraan kasad th asian taekwondo open championship kejuaraan internasional diselenggarakan pbti .,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Keyword Extraction

In [11]:
# Menghitung jumlah kemunculan pada tiap kata
word_frequencies = df.sum(axis=0)

# Mengurutkan berdasarkan jumlah kemunculan kata
sorted_word_frequencies = word_frequencies.sort_values(ascending=False)

# Menampilkan kata teratas
keyword = 3

for i, (word, freq) in enumerate(sorted_word_frequencies.items()):
    if i < keyword:
        print(f"Rank {i+1}: {word} (Frequency: {freq})")

Rank 1: . (Frequency: 15)
Rank 2: taekwondo (Frequency: 12)
Rank 3: kejuaraan (Frequency: 9)
